# Configuring CPU frequency scaling

This Notebook demonstrates how to set [CPU frequency](https://wiki.archlinux.org/index.php/CPU_frequency_scaling#Setting_maximum_and_minimum_frequencies) manually. This also illustrates how you can adjust kernel boot parameters to enable/disable certain non-default features of the underlying Linux kernel within your disk image.

## Variables used in this notebook

In [ ]:
OS_PROJECT_NAME="$OS_PROJECT_NAME"
OS_REGION_NAME="$OS_REGION_NAME"
NODE_TYPE=compute_skylake
IMAGE=CC-CentOS7

# A unique name for most provisioned resources to avoid collisions
RESOURCE_NAME="${USER}-cpufreq-$(date +%b%d)"

## Reserve resources

The following script will create a reservation for one node of type `$NODE_TYPE`, plus a public IP address.

In [ ]:
lease_name="$RESOURCE_NAME"
network_name="$RESOURCE_NAME"
public_network_id=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation resource_type=virtual:floatingip,network_id="$public_network_id",amount=1 \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$lease_name"

# Wait for lease to start
timeout 300 bash -c 'until [[ $(blazar lease-show $0 -f value -c status) == "ACTIVE" ]]; do sleep 1; done' "$lease_name" \
    && echo "Lease started successfully!"

#
# Fetch information about which resources were reserved for later use
#

reservations=$(blazar lease-show "$lease_name" -f json \
  | jq -r '.reservations')
host_reservation_id=$(jq -rs 'map(select(.resource_type=="physical:host"))[].id' <<<"$reservations")
fip_reservation_id=$(jq -rs 'map(select(.resource_type=="virtual:floatingip"))[].id' <<<"$reservations")

fip=$(openstack floating ip list --tags "reservation:$fip_reservation_id" -f json)
fip_id=$(jq -r 'map(.ID)[0]' <<<"$fip")
fip_addr=$(jq -r 'map(.["Floating IP Address"])[0]' <<<"$fip")

### Provision resources

In [ ]:
key_pair_upload

# Create the server instance
openstack server create --wait \
 --flavor baremetal \
 --image "$IMAGE" \
 --hint reservation="$host_reservation_id" \
 --nic net-id=$(openstack network show sharednet1 -f value -c id) \
 --key-name "$USER-jupyter" \
 "$RESOURCE_NAME"
 
# Assign a public IP
openstack server add floating ip "$RESOURCE_NAME" "$fip_addr"

# Wait for public IP connectivity
wait_ssh "$fip_addr"

# Create a helper function to run commands remotely
server() {
  quoted="$(printf "%q " "$@")"
  ssh cc@"$fip_addr" bash -c "'$quoted'"
}
export -f server

## Introduction to CPU frequency interface

The [`cpupower`](https://wiki.archlinux.org/index.php/CPU_frequency_scaling#cpupower) binary can be used to influence the state of the CPU frequency scaling. Notice that by default the `driver` is set to `intel_pstate`.

In [ ]:
# Get current (stock) CPU frequency configuration
server cpupower frequency-info

This is actually a problem for us, because we want to set the CPU frequency to a fixed value defined by us. The `intel_pstate` driver only supports two [governors](https://www.kernel.org/doc/Documentation/cpu-freq/governors.txt): `performance` and `powersave`. In order to proceed, we need to update our kernel configuration to disable the `intel_pstate` driver. We can do this by [modifying the Grub2 configuration](https://chameleoncloud.readthedocs.io/en/latest/technical/baremetal.html#customizing-the-kernel).

In [ ]:
server sudo sed -i.bak 's/\(GRUB_CMDLINE_LINUX_DEFAULT=.*\)"$/\1 intel_pstate=disable"/g' /etc/default/grub
server sudo grub2-mkconfig -o /boot/grub2/grub.cfg

We now need to reboot the server for the kernel change to take effect.

In [ ]:
openstack server reboot "$RESOURCE_NAME" --wait
# Wait for boot to complete
wait_ssh "$fip_addr"

After a reboot, we should (hopefully) see that the driver has now defaulted to `acpi-cpufreq`. Notice also that the `userspace` governor is now available.

In [ ]:
server cpupower frequency-info

## Set fixed CPU frequency

First, enable the "userspace" governor, which allows us to specify a fixed value.

In [ ]:
server sudo cpupower frequency-set -g userspace >/dev/null

Then, set the fixed value.

In [ ]:
server sudo cpupower frequency-set -f "2.10GHz" >/dev/null
server cpupower frequency-info

The CPU frequency has now been set to a fixed value. You can make further changes to the value using the `cpupower` tool.